In [1]:
import os
os.environ["CHAINER_TYPE_CHECK"] = "0"

import math
import random
import numpy as np
import chainer
from chainer import cuda, Chain, optimizers, Variable, serializers, Link
import chainer.functions as F
import chainer.links as L
import cupy
import cv2

import time

from custom.loader4 import MultiLoader
from custom.loader_senga4 import MultiLoaderSenga
from custom.net import Generator, Discriminator, copy_param, soft_copy_param
from custom.loss import G_loss, D_loss
from custom.pggan_inception import calc_inception
import custom.config

#import matplotlib
import matplotlib.pyplot as plt

In [2]:
xp = cuda.cupy
#print(chainer.functions.Linear(1,1).type_check_enable)

In [3]:
gpu = custom.config.train_params['gpu']
init=custom.config.train_params['init_iter']
batches = custom.config.train_params['batches']
smoothing = custom.config.train_params['smoothing']
mirror = custom.config.train_params['image_mirror']
test_interval = custom.config.train_params['test_interval']
save_interval = custom.config.train_params['save_interval']
max_stage = custom.config.train_params['max_stage']
inception_interval = custom.config.train_params['inception_interval']
inception = custom.config.train_params['inception']
# Genの学習のzを切り替えるかどうか
separate = custom.config.train_params['separate']
# Minibatchリピート回数
minibatch_repeats = custom.config.train_params['minibatch_repeats']
senga = custom.config.train_params['senga']
# stage学習枚数
interval = custom.config.train_params['interval']
n_hidden = custom.config.network_params['latent_in']
print(custom.config.train_params)
print(custom.config.network_params)
print(custom.config.loss_params)

{'separate': True, 'inception_interval': 2500, 'senga': False, 'test_interval': 100, 'smoothing': 0.999, 'inception': False, 'gpu': 1, 'init_iter': 0, 'image_mirror': False, 'minibatch_repeats': 4, 'width': 512, 'max_stage': 14, 'save_interval': 5000, 'height': 512, 'interval': 400000, 'batches': (16, 16, 16, 16, 16, 16, 8, 2)}
{'image_ch': 3, 'D_filters': (32, 64, 128, 256, 512, 512, 512, 512), 'latent_in': 512, 'slope': 0.2, 'G_filters': (512, 512, 512, 512, 256, 128, 64, 32)}
{'iwass': 1.0}


In [4]:
gen = Generator(n_hidden=n_hidden, max_stage=max_stage)
generator_smooth = Generator(n_hidden=n_hidden, max_stage=max_stage)
if init != 0:
    serializers.load_npz("generator.npz", gen)
else:
    serializers.load_npz("./model/generator.npz", gen)
    

gen.compute_accuracy = False
generator_smooth.compute_accuracy = False


chainer.cuda.get_device(gpu).use()

gen.to_gpu(gpu) # GPUを使うための処理
generator_smooth.to_gpu(gpu) # GPUを使うための処理

# コピー
copy_param(generator_smooth, gen)

if init != 0:
    serializers.load_npz("generator_smooth.npz", generator_smooth)
else:
    serializers.load_npz("./model/generator_smooth.npz", generator_smooth)

gen_optimizer = optimizers.Adam(alpha=0.001, beta1=0.0, beta2=0.99)
#gen_optimizer = optimizers.Adam(alpha=0.01, beta1=0.0, beta2=0.99)
gen_optimizer.setup(gen)



# Critic
cri = Discriminator()
cri.compute_accuracy = False
if init != 0:
    serializers.load_npz("critic.npz", cri)
else:
    serializers.load_npz("./model/critic.npz", cri)
    
cri.to_gpu(gpu) # GPUを使うための処理

cri_optimizer = optimizers.Adam(alpha=0.001, beta1=0.0, beta2=0.99) # 0.001
#cri_optimizer = optimizers.Adam(alpha=0.01, beta1=0.0, beta2=0.99)

cri_optimizer.setup(cri)
#cri_optimizer.add_hook(chainer.optimizer.WeightDecay(0.0001)) # 0.005




ValueError: could not broadcast input array from shape (32,32,3,3) into shape (64,64,3,3)

In [ ]:
if not senga:
    mloader = custom.loader4.MultiLoader(passA='./train',passB='')
    total_num = mloader.num
    mloader.fileListB
else:
    mloader = custom.loader_senga4.MultiLoaderSenga(passA='./train_not_senga',passB='')
    total_num = mloader.num
    mloader.fileListB

In [ ]:
def real_image(images, stage):
    size = config.width

    if math.floor(stage)%2==0:
        reso = min(size, 4 * 2**(((math.floor(stage)+1)//2)))
        scale = max(1, size//reso)
        if scale>1:
            images = F.average_pooling_2d(images, scale, scale, 0)
    else:
        alpha = stage - math.floor(stage)
        reso_low = min(size, 4 * 2**(((math.floor(stage))//2)))
        reso_high = min(size, 4 * 2**(((math.floor(stage)+1)//2)))
        scale_low = max(1, size//reso_low)
        scale_high = max(1, size//reso_high)
        
        images_low = F.unpooling_2d(F.average_pooling_2d(x_real, scale_low, scale_low, 0), 2, 2, 0, outsize=(reso_high, reso_high))
        
        if scale_high > 1:
            images_high = F.average_pooling_2d(images, scale_high, scale_high, 0)
        else:
            images_high = images
        images = (1.0-alpha)*images_low + alpha*images_high
        
    return images

In [ ]:
def real_image2(images, stage):


    if math.floor(stage)%2==0:
        return images
    else:
        h,w = images.shape[2:]
        alpha = stage - math.floor(stage)
        images_low = F.unpooling_2d(F.average_pooling_2d(images, 2, 2, 0), 2, 2, 0, outsize=(h, w))
        images = (1.-alpha)*images_low + alpha*images
        
    return images

In [ ]:
def getCount2(init, batches, interval):
    counter = 0
    hoge = init
    for i in range(len(batches)*2):
        tmp = interval / batches[(i+1)//2]

        hoge = hoge - tmp
        
        if hoge > 0:
            counter += interval
        else:
            hoge = hoge + tmp
            counter += hoge*batches[(i+1)//2]
            break
    return (int)(counter)

In [ ]:
print(total_num)

In [ ]:
# シード値設定
np.random.seed(seed=20180101)
xp.random.seed(seed=20180101)
print(sum(p.data.size for p in gen.params()))
print(sum(p.data.size for p in cri.params()))

In [ ]:
n_critic = 1

g_loss_data = 0
d_loss_data = 0
w_dis = 0
w_dis_sum = 0

grad_loss_data = 0

zTest = generator_smooth.make_hidden(16)
zTest = Variable(zTest)

stage=0
if os.path.isfile('stage.txt') and init!=0:
    stage = np.loadtxt('stage.txt')
# w_dis保存用
w_dis_list = np.empty((0,2))
if os.path.isfile('log.csv') and init!=0:
    w_dis_list = np.loadtxt('log.csv', delimiter=',')

# 時間計測
A = time.time()


counter = getCount2(init, batches, interval)

kimg_list = []
inception_list = []

pre_stage = -1

for i in range(10000000):
    
    
    if (init+i) % minibatch_repeats == 0 or i == 0:
        stage = counter / interval
    #stage = (init+i) / 100000.0
    
    if stage > max_stage:
        stage = max_stage
        
    # adam_reset
    if (np.floor(stage) != np.floor(pre_stage) or np.ceil(stage) != np.ceil(pre_stage)) and pre_stage != -1:
        print("reset optimizer")
        gen_optimizer = optimizers.Adam(alpha=0.001, beta1=0.0, beta2=0.99)
        gen_optimizer.setup(gen)
        
        cri_optimizer = optimizers.Adam(alpha=0.001, beta1=0.0, beta2=0.99) # 0.001
        cri_optimizer.setup(cri)
    pre_stage = stage
    
    batch_size = batches[int((stage+1)//2)]
    #stage = 11.5
    # critic更新
    
    if (init+i) % minibatch_repeats == 0 or i == 0:
        # 本物用
        size = 4*2**((int(stage)+1)//2)
        x_real = mloader.getBatch(batch_size=batch_size, width=size, height=size, alpha=1.0, mirror=mirror)
        x_real = Variable(xp.asarray(x_real, dtype=np.float32))

        # スケール合わせ
        x_real = real_image2(x_real, stage)
    
    z = gen.make_hidden(batch_size)
    z = Variable(z)
    
    # d_loss
    d_w_loss, grad_loss, drift_loss, wass = D_loss(G=gen, D=cri, stage=stage, reals=x_real, latents=z, batch_size=batch_size)
    
    d_loss = d_w_loss+grad_loss+drift_loss
    
    w_dis += wass
    w_dis_sum += wass
    
    d_loss_data += d_w_loss.data
    grad_loss_data+=grad_loss.data

    cri.cleargrads()
    d_loss.backward()
    cri_optimizer.update()

    # BackPropの参照を削除
    d_loss.unchain_backward()
    
    # Genの学習のzを切り替えるかどうか
    if separate:
        
        # 偽物用
        z = gen.make_hidden(batch_size)
        z = Variable(z)
    
    g_loss = G_loss(G=gen, D=cri, stage=stage, latents=z, batch_size=batch_size)
    g_loss_data += g_loss.data
    
    gen.cleargrads()
    g_loss.backward()
    gen_optimizer.update()
    
    # BackPropの参照を削除
    g_loss.unchain_backward()
    
    # update smoothed generator
    soft_copy_param(generator_smooth, gen, 1.0-smoothing)
    
    if (i+init) % save_interval == 0 and i != 0:
        print("save")
        gen.to_cpu()
        serializers.save_npz("generator.npz", gen)
        gen.to_gpu(gpu)
        
        generator_smooth.to_cpu()
        serializers.save_npz("generator_smooth.npz", generator_smooth)
        generator_smooth.to_gpu(gpu)
        
        cri.to_cpu()
        serializers.save_npz("critic.npz", cri)
        cri.to_gpu(gpu)
        # 現在のステージを保存
        np.savetxt('stage.txt', np.array([stage]))
        print("ok")
        
    if i % 500 == 0 and i != 0:
        # w_dis保存
        w_dis_list = np.append(w_dis_list, np.array([[int(init+i), w_dis_sum/500]]), axis=0)
        np.savetxt('log.csv', w_dis_list,delimiter=',')
        w_dis_sum = 0
        
    
    if inception and i % inception_interval == 0:
        # inception
        print("calc inception...")
        mean, std = calc_inception(gen, 2500, stage, gpu)
        print(mean, std)
        inception_list.append(mean)
        
        kimg_list.append((init+i)/1000.0)
        #x = np.arange(i+1)
        #print()
        plt.plot(np.asarray(kimg_list), np.asarray(inception_list))
        plt.xlabel('iter (10^3)')
        plt.ylabel('inception score')
        plt.savefig('inception.jpg')
        plt.close()
    
    if (init+i) % 20000 == 0 and i != 0:
        print("save")
        gen.to_cpu()
        serializers.save_npz("./weight/generator_" + str(init+i) + ".npz", gen)
        gen.to_gpu(gpu)
        
        generator_smooth.to_cpu()
        serializers.save_npz("./weight/generator_smooth_" + str(init+i) + ".npz", generator_smooth)
        generator_smooth.to_gpu(gpu)
        
        cri.to_cpu()
        serializers.save_npz("./weight/critic_" + str(init+i) + ".npz", cri)
        cri.to_gpu(gpu)
        # 現在のステージを保存
        np.savetxt('stage.txt', np.array([stage]))
        print("ok")
    
    if i % test_interval == 0:

        # 画像出力
        # テストモードに
        with chainer.using_config('train', False), chainer.no_backprop_mode():
            
            # 偽物用
            x_test = generator_smooth(zTest, stage)
            #y_test = cri(x_test)
            
            size = 4*2**((int(stage)+1)//2)
            
            print(str(init+i) + ",epoch:" + str(int(batch_size/total_num*(init+i)/minibatch_repeats)) + ",gen:" + str(g_loss_data/test_interval) + 
                  ", cri:" + str(d_loss_data/test_interval) + ", w_dis:" + str(w_dis/test_interval) + ", grad_loss:" + str(grad_loss_data/test_interval))
            print("stage:" + str(stage) + ", size:" + str(size) + ", counter:" + str(counter) + ", batch:" + str(batch_size))
            
            g_loss_data = 0
            d_loss_data = 0
            w_dis = 0
            grad_loss_data = 0
            
        imgs = cuda.to_cpu(x_test.data)
        imgs = np.transpose(imgs, (0, 3, 2, 1))

        # 画像を連結
        img2 = []
        
        s = 4

        for k in range(s):
            img2.append(imgs[k*s])

            for j in range(s):
                if j != 0:
                    img2[k] = cv2.hconcat([img2[k], imgs[k*s+j]])
            if k == 0:
                img3 = img2[0]
            else:
                img3 = cv2.vconcat([img3, img2[k]])

        img3 = np.asarray(np.clip(img3 * 127.5 + 127.5, 0.0, 255.0), dtype=np.uint8)
        cv2.imwrite('./res/'+str(init+i)+'.jpg', img3[:,:,:3])
        
        # かかった時間
        print("time:{0} sec".format(time.time()-A))
        
        A = time.time()
        
    counter+=batch_size 

In [ ]:
gen.to_cpu()
serializers.save_npz("generator.npz", gen)
cri.to_cpu()
serializers.save_npz("critic.npz", cri)